In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('main_task.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [4]:
df

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


### Обработка NaN

In [5]:
# Заполним пропущенные значения Reviews пробелами, чтобы не вызвать ошибки на следующем шаге
df['Reviews'] = df['Reviews'].fillna(' ')

In [6]:
# Заполним пропущенные значения Number of Reviews так: 0 - если при этом нет Reviews, и 1 если есть. Такой подход выбран
# исходя из данных - у пропущенных значений Number of Reviews информация об отзывах либо также пропущена, либо имеется 1 отзыв 

def fill_no_of_reviews (row):
    if pd.isna(row['Number of Reviews']):
        if len(row['Reviews']) > 8:         # длина Reviews, являющаяся пропуском [[], []]
            return 1
        return 0
    return row['Number of Reviews']
    
df['Number of Reviews'] = df.apply(lambda row: fill_no_of_reviews(row), axis=1) 

### Feature Engineering

In [7]:
# 1. Добавим ординальный признак уровня цен со значениями от 1 до 3. Проаущенные значения заменяем нулями.

def price_level (row):
    if pd.isna(row['Price Range']):
        return 0
    elif row['Price Range']=='$':
        return 1
    elif row['Price Range']=='$$ - $$$':
        return 2
    elif row['Price Range']=='$$$$':
        return 3
        
df['price_level'] = df.apply(lambda row: price_level (row), axis = 1)

In [8]:
# 2. Добавляем признак = количество представленных кухонь в ресторане

df['cuisine_count'] = df['Cuisine Style'].str[2:-2].str.split("', '").fillna('1').str.len()

In [9]:
# 3. Добавляем категорийный признак присутствия каждой кухни

cuisine_list = df['Cuisine Style'].str[2:-2].str.split("', '").dropna().explode().unique()
for cuisine in cuisine_list:
    df[cuisine] = df['Cuisine Style'].fillna('none').apply(lambda x: 1 if x.find(cuisine)>-1 else 0)

In [10]:
# 4. Добавим категорийный признак того, что город популярное тур.направление

tourism_dst = ['Amsterdam','Athens','Barcelona','Berlin','Dublin','Lisbon','London','Madrid','Milan','Munich','Paris','Prague','Rome','Vienna']
df['tourism_dst'] = df['City'].apply(lambda x: 1 if x in tourism_dst else 0 )

In [11]:
# 5. Добавим категорийный признак того, что город - столица

capital_city = ['Amsterdam','Athens','Berlin','Bratislava','Brussels','Budapest','Copenhagen','Dublin','Edinburgh','Helsinki','Lisbon','Ljubljana','London','Luxembourg','Madrid','Oslo','Paris','Prague','Rome','Stockholm','Vienna','Warsaw']
df['capital_city'] = df['City'].apply(lambda x: 1 if x in capital_city else 0 )

In [12]:
# 6. Добавляем категорийный признак принадлежности городу

df['City_temp'] = df['City']
df = pd.get_dummies(df, columns=['City'], dummy_na=True)
df = df.rename(columns = {'City_temp': 'City'})

In [13]:
# 7. Добавляем признак = разница в днях между последним и предыдущим отзывом 

df['Date_1'] = df['Reviews'].str.findall(r'\d+\W\d+\W\d+').str.get(0)
df['Date_1'] =pd.to_datetime(df['Date_1'], errors='coerce')
df['Date_2'] = df['Reviews'].str.findall(r'\d+\W\d+\W\d+').str.get(1)
df['Date_2'] =pd.to_datetime(df['Date_2'], errors='coerce')
df['rev_dates_difference'] = (df['Date_1'] - df['Date_2'])
df['rev_dates_difference'] = df['rev_dates_difference'].astype('timedelta64[D]').fillna(0)

df.drop(['Date_1','Date_2'], axis = 1, inplace=True)

In [14]:
# Удаляем столбцы с данными типа object

object_columns = [s for s in df.columns if df[s].dtypes == 'object']
df.drop(object_columns, axis = 1, inplace=True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 165 entries, Ranking to rev_dates_difference
dtypes: float64(4), int64(129), uint8(32)
memory usage: 41.8 MB


### Создаём, обучаем и тестируем модель

In [16]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
X = df.drop(['Rating'], axis = 1)  
y = df['Rating']  
  
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
  
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

In [17]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
  
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
  
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
# округлим до половины целого, т.к. рейтинг на TripAdvisor идет с шагом 0.5
y_pred = np.round(regr.predict(X_test) * 2) / 2

In [18]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.

print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.181
